# Supplement Figures: S11-14

end of ms/redo_rh_scaling.ipynb

In [ ]:
sys.path.append("/glade/u/home/bbuchovecky/projects/pme_ppe")
import pme_ppe_utils as ppe

## Get plotting variables
plt.rcParams.update({
    'figure.dpi': 150,
    'font.family': 'sans-serif',
    'font.size': 8,
    'font.weight': 'normal',
    'axes.labelsize': myplt.axlabelsize,
    'axes.labelweight': 'normal',
    'xtick.direction': 'out',
    'ytick.direction': 'out',
    'xtick.major.size': 4,
    'ytick.major.size': 4,
    'legend.fontsize': myplt.legendsize,
    'legend.frameon': False,
    'hatch.linewidth': 0.2,
})

## Load model output

# Get area weights
cesm_weights = xr.open_dataset("/glade/work/bbuchovecky/PPE_analysis/weights/COUP_PPE_weights_atmgrid.nc", decode_timedelta=True).rename({"globweights": "globalweights"})
assert cesm_weights.landweights.sum(dim=["lat", "lon"]) == 1

hadcm_weights = xr.open_dataset("/glade/work/bbuchovecky/HadCM3_analysis/weights/HadCM3_weights_v2.nc", decode_timedelta=True)
assert hadcm_weights.landweights.sum(dim=["lat", "lon"]) == 1

# Identify variables to load
varlist = ["T_S", "P", "ET", "RH_S", ]

# Load time-mean variables
cesm_ppe, cesm_ctl = ppe.get_cesm(varlist, "time_mean")
hadhst_ppe, hadhst_ctl = ppe.get_hadcm(varlist, "control", "time_mean")
hada1b_ppe, hada1b_ctl = ppe.get_hadcm(varlist, "a1b", "time_mean")
dict_cmip_co2, dict_cmip_pi, dict_cmip_weights = ppe.get_cmip(varlist, "time_mean")

# Store list of CMIP model names
cmip_models = list(dict_cmip_co2.keys())

# Convert units if necessary
units = "mm/day"

if units == "W/m2":
    mmday_to_Wm2 =  2.26e6 / (60 * 60 * 24)

    # CESM2 and HadCM3
    for v in ["P", "ET"]:
        if v in varlist:
            cesm_ppe[v] = cesm_ppe[v] * mmday_to_Wm2
            cesm_ctl[v] = cesm_ctl[v] * mmday_to_Wm2
            hadhst_ppe[v] = hadhst_ppe[v] * mmday_to_Wm2
            hadhst_ctl[v] = hadhst_ctl[v] * mmday_to_Wm2
            hada1b_ppe[v] = hada1b_ppe[v] * mmday_to_Wm2
            hada1b_ctl[v] = hada1b_ctl[v] * mmday_to_Wm2

    # CMIP
    for m in cmip_models:
        for v in ["P", "ET"]:
            if v in varlist:
                dict_cmip_co2[m][v] = dict_cmip_co2[m][v] * mmday_to_Wm2
                dict_cmip_pi[m][v] = dict_cmip_pi[m][v] * mmday_to_Wm2

print(f"water flux units = {units}")


# Compute the land and ocean mean
cesm_ppe_areamean = dict()
hadhst_ppe_areamean = dict()
hada1b_ppe_areamean = dict()
cmip_co2_areamean = dict()

cesm_ctl_areamean = dict()
hadhst_ctl_areamean = dict()
hada1b_ctl_areamean = dict()
cmip_pi_areamean = dict()

cesm_delta_areamean = dict()
hadhst_delta_areamean = dict()
hada1b_delta_areamean = dict()
cmip_delta_areamean = dict()
cmip_areamean = dict()

for w, weight in zip(["lm", "om"], ["landweights", "oceanweights"]):
    try:
        cesm_ppe_areamean[w];
    except KeyError:
        cesm_ppe_areamean[w] = dict()
        hadhst_ppe_areamean[w] = dict()
        hada1b_ppe_areamean[w] = dict()
        cmip_co2_areamean[w] = dict()

        cesm_ctl_areamean[w] = dict()
        hadhst_ctl_areamean[w] = dict()
        hada1b_ctl_areamean[w] = dict()
        cmip_pi_areamean[w] = dict()

        cesm_delta_areamean[w] = dict()
        hadhst_delta_areamean[w] = dict()
        hada1b_delta_areamean[w] = dict()
        cmip_delta_areamean[w] = dict()
        cmip_areamean[w] = dict()

    assert np.round(cesm_weights[weight].sum(), 3) == 1.
    assert np.round(hadcm_weights[weight].sum(), 3) == 1.

    for v in varlist:
        cesm_ppe_areamean[w][v] = cesm_ppe[v].weighted(cesm_weights[weight]).mean(dim=["lat", "lon"])
        hadhst_ppe_areamean[w][v] = hadhst_ppe[v].weighted(hadcm_weights[weight]).mean(dim=["lat", "lon"])
        hada1b_ppe_areamean[w][v] = hada1b_ppe[v].weighted(hadcm_weights[weight]).mean(dim=["lat", "lon"])
        
        cesm_ctl_areamean[w][v] = cesm_ctl[v].weighted(cesm_weights[weight]).mean(dim=["lat", "lon"])
        hadhst_ctl_areamean[w][v] = hadhst_ctl[v].weighted(hadcm_weights[weight]).mean(dim=["lat", "lon"])
        hada1b_ctl_areamean[w][v] = hada1b_ctl[v].weighted(hadcm_weights[weight]).mean(dim=["lat", "lon"])

        cesm_delta_areamean[w][v] = (cesm_ppe[v] - cesm_ctl[v]).weighted(cesm_weights[weight]).mean(dim=["lat", "lon"])
        hadhst_delta_areamean[w][v] = (hadhst_ppe[v] - hadhst_ctl[v]).weighted(hadcm_weights[weight]).mean(dim=["lat", "lon"])
        hada1b_delta_areamean[w][v] = (hada1b_ppe[v] - hada1b_ctl[v]).weighted(hadcm_weights[weight]).mean(dim=["lat", "lon"])

        cmip_co2_list = []
        cmip_pi_list = []
        cmip_delta_list = []
        cmip_list = []
        for m in cmip_models:
            assert np.round(dict_cmip_weights[m][weight].sum(), 4) == 1.
            co2 = dict_cmip_co2[m][v].weighted(dict_cmip_weights[m][weight]).mean(dim=["lat", "lon"])
            pi = dict_cmip_pi[m][v].weighted(dict_cmip_weights[m][weight]).mean(dim=["lat", "lon"])
            cmip_co2_list.append(co2)
            cmip_pi_list.append(pi)
            cmip_delta_list.append(co2 - pi)
            cmip_list.append(xr.concat([pi.assign_coords(experiment=0),co2.assign_coords(experiment=[1,2,3])], dim="experiment"))
        cmip_co2_areamean[w][v] = xr.concat(cmip_co2_list, dim='model', coords='minimal', compat='override').assign_coords(model=("model", np.arange(7)), model_name=("model", cmip_models))
        cmip_pi_areamean[w][v] = xr.concat(cmip_pi_list, dim='model', coords='minimal', compat='override').assign_coords(model=("model", np.arange(7)), model_name=("model", cmip_models))
        cmip_delta_areamean[w][v] = xr.concat(cmip_delta_list, dim='model', coords='minimal', compat='override').assign_coords(model=("model", np.arange(7)), model_name=("model", cmip_models))
        cmip_areamean[w][v] = xr.concat(cmip_list, dim='model', coords='minimal', compat='override').assign_coords(model=("model", np.arange(7)), model_name=("model", cmip_models))

In [ ]:
delta_areamean_lat = dict()

lat_bounds = {"global": 90, "trop": 30}

for key, lat in lat_bounds.items():
    delta_areamean_lat[key] = dict()

    for name, ppe, ctl, weights in zip(
        ["cesm", "hadhst", "hada1b"],
        [cesm_ppe, hadhst_ppe, hada1b_ppe],
        [cesm_ctl, hadhst_ctl, hada1b_ctl],
        [cesm_weights.fillna(0), hadcm_weights.fillna(0), hadcm_weights.fillna(0)]):

        ppe_P_L = ppe["P"].sortby("lat").sel(lat=slice(-lat, lat)).weighted(weights.landweights).mean(dim=["lat", "lon"])
        ppe_P_O = ppe["P"].sortby("lat").sel(lat=slice(-lat, lat)).weighted(weights.oceanweights).mean(dim=["lat", "lon"])
        ppe_RH_SL = ppe["RH_S"].sortby("lat").sel(lat=slice(-lat, lat)).weighted(weights.landweights).mean(dim=["lat", "lon"]) / 100
        ppe_RH_SO = ppe["RH_S"].sortby("lat").sel(lat=slice(-lat, lat)).weighted(weights.oceanweights).mean(dim=["lat", "lon"]) / 100
        ctl_P_L = ctl["P"].sortby("lat").sel(lat=slice(-lat, lat)).weighted(weights.landweights).mean(dim=["lat", "lon"])
        ctl_P_O = ctl["P"].sortby("lat").sel(lat=slice(-lat, lat)).weighted(weights.oceanweights).mean(dim=["lat", "lon"])
        ctl_RH_SL = ctl["RH_S"].sortby("lat").sel(lat=slice(-lat, lat)).weighted(weights.landweights).mean(dim=["lat", "lon"]) / 100
        ctl_RH_SO = ctl["RH_S"].sortby("lat").sel(lat=slice(-lat, lat)).weighted(weights.oceanweights).mean(dim=["lat", "lon"]) / 100

        ppe_P_L = ppe_P_L.sortby("member")
        ppe_P_O = ppe_P_O.sortby("member")
        ppe_RH_SL = ppe_RH_SL.sortby("member")
        ppe_RH_SO = ppe_RH_SO.sortby("member")

        delta_P_L = ppe_P_L - ctl_P_L
        delta_RH_SL = ppe_RH_SL - ctl_RH_SL
        delta_RH_SO = ppe_RH_SO - ctl_RH_SO

        delta_areamean_lat[key][name] = dict()
        delta_areamean_lat[key][name]["P_L"] = ppe_P_L - ctl_P_L
        delta_areamean_lat[key][name]["P_O"] = ppe_P_O - ctl_P_O
        delta_areamean_lat[key][name]["RH_SL"] = ppe_RH_SL - ctl_RH_SL
        delta_areamean_lat[key][name]["RH_SO"] = ppe_RH_SO - ctl_RH_SO

In [ ]:
def compute_delta_p_scaling(a_1, rh_l_pert, rh_o_pert, p_o_pert, rh_l_ctrl, rh_o_ctrl, p_o_ctrl):
    return a_1 * (((rh_l_pert - rh_o_pert) * p_o_pert) - ((rh_l_ctrl - rh_o_ctrl) * p_o_ctrl))

def compute_approx_delta_p_scaling(a_1, rh_l_pert, rh_o_pert, rh_l_ctrl, rh_o_ctrl, p_o_ctrl):
    return a_1 * (((rh_l_pert - rh_o_pert) * p_o_ctrl) - ((rh_l_ctrl - rh_o_ctrl) * p_o_ctrl))

delta_p_scaling = dict()

lat_bounds = {"global": 90, "trop": 30}

for key, lat in lat_bounds.items():
    delta_p_scaling[key] = dict()

    for name, ppe, ctl, weights in zip(
        ["cesm", "hadhst", "hada1b"],
        [cesm_ppe, hadhst_ppe, hada1b_ppe],
        [cesm_ctl, hadhst_ctl, hada1b_ctl],
        [cesm_weights.fillna(0), hadcm_weights.fillna(0), hadcm_weights.fillna(0)]):

        ppe_P_L = ppe["P"].sortby("lat").sel(lat=slice(-lat, lat)).weighted(weights.landweights).mean(dim=["lat", "lon"])
        ppe_P_O = ppe["P"].sortby("lat").sel(lat=slice(-lat, lat)).weighted(weights.oceanweights).mean(dim=["lat", "lon"])
        ppe_RH_SL = ppe["RH_S"].sortby("lat").sel(lat=slice(-lat, lat)).weighted(weights.landweights).mean(dim=["lat", "lon"]) / 100
        ppe_RH_SO = ppe["RH_S"].sortby("lat").sel(lat=slice(-lat, lat)).weighted(weights.oceanweights).mean(dim=["lat", "lon"]) / 100
        ctl_P_L = ctl["P"].sortby("lat").sel(lat=slice(-lat, lat)).weighted(weights.landweights).mean(dim=["lat", "lon"])
        ctl_P_O = ctl["P"].sortby("lat").sel(lat=slice(-lat, lat)).weighted(weights.oceanweights).mean(dim=["lat", "lon"])
        ctl_RH_SL = ctl["RH_S"].sortby("lat").sel(lat=slice(-lat, lat)).weighted(weights.landweights).mean(dim=["lat", "lon"]) / 100
        ctl_RH_SO = ctl["RH_S"].sortby("lat").sel(lat=slice(-lat, lat)).weighted(weights.oceanweights).mean(dim=["lat", "lon"]) / 100

        ppe_P_L = ppe_P_L.sortby("member")
        ppe_P_O = ppe_P_O.sortby("member")
        ppe_RH_SL = ppe_RH_SL.sortby("member")
        ppe_RH_SO = ppe_RH_SO.sortby("member")

        delta_P_L = ppe_P_L - ctl_P_L
        delta_RH_SL = ppe_RH_SL - ctl_RH_SL
        delta_RH_SO = ppe_RH_SO - ctl_RH_SO

        _delta_p_scaling = compute_delta_p_scaling(
            lr[key][name]["lr"].slope, ppe_RH_SL, ppe_RH_SO, ppe_P_O, ctl_RH_SL, ctl_RH_SO, ctl_P_O,
        )

        _delta_p_scaling = compute_approx_delta_p_scaling(
            lr[key][name]["lr"].slope, ppe_RH_SL, ppe_RH_SO, ctl_RH_SL, ctl_RH_SO, ctl_P_O,
        )

        slope, intercept, rvalue, pvalue, stderr = stats.linregress(delta_P_L, _delta_p_scaling)
        rmse = np.sqrt(((delta_P_L - _delta_p_scaling) ** 2).mean())
        full = dict(
            data=_delta_p_scaling,
            slope=slope,
            intercept=intercept,
            r_value=rvalue,
            p_value=pvalue,
            std_err=stderr,
            rmse=rmse.item(),
        )

        slope, intercept, rvalue, pvalue, stderr = stats.linregress(delta_P_L, _delta_p_scaling)
        rmse = np.sqrt(((delta_P_L - _delta_p_scaling) ** 2).mean())
        full_approx = dict(
            data=_delta_p_scaling,
            slope=slope,
            intercept=intercept,
            r_value=rvalue,
            p_value=pvalue,
            std_err=stderr,
            rmse=rmse.item(),
        )

        delta_rh_l_term = lr[key][name]["lr"].slope * delta_RH_SL * ctl_P_O
        slope, intercept, rvalue, pvalue, stderr = stats.linregress(delta_P_L, delta_rh_l_term)
        rmse = np.sqrt(((delta_P_L - delta_rh_l_term) ** 2).mean())
        delta_rh_l_term = dict(
            data=delta_rh_l_term,
            slope=slope,
            intercept=intercept,
            r_value=rvalue,
            p_value=pvalue,
            std_err=stderr,
            rmse=rmse.item(),
        )

        delta_rh_o_term = lr[key][name]["lr"].slope * delta_RH_SO * ctl_P_O
        slope, intercept, rvalue, pvalue, stderr = stats.linregress(delta_P_L, -delta_rh_o_term)
        rmse = np.sqrt(((delta_P_L - delta_rh_o_term) ** 2).mean())
        delta_rh_o_term = dict(
            data=delta_rh_o_term,
            slope=slope,
            intercept=intercept,
            r_value=rvalue,
            p_value=pvalue,
            std_err=stderr,
            rmse=rmse.item(),
        )

        delta_p_scaling[key][name] = dict(full=full, full_approx=full_approx, delta_rh_l_term=delta_rh_l_term, delta_rh_o_term=delta_rh_o_term)

## S11: One-to-one scatter plots of actual and diagnosed \delta(P-E) over global land from the moisture budget scaling for the CESM2 PI PPE

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(13, 7), sharey=True, sharex=False, layout="constrained")
ax = axs.flatten()


ax[0].scatter(delta_P_lm, delta_E_lm, color=myplt.c_dict["cesm"], s=myplt.s_dict["cesm"], marker=myplt.m_dict["cesm"])
ax[1].scatter(delta_P_lm, term1_lm+term2_lm+term3_lm+term4_lm+term5_lm, color=myplt.c_dict["cesm"], s=myplt.s_dict["cesm"], marker=myplt.m_dict["cesm"])
ax[2].scatter(delta_P_lm, delta_E_lm+term1_lm+term2_lm+term3_lm+term4_lm+term5_lm, color=myplt.c_dict["cesm"], s=myplt.s_dict["cesm"], marker=myplt.m_dict["cesm"])

ax[3].scatter(delta_P_lm, term1_lm, color=myplt.c_dict["cesm"], s=myplt.s_dict["cesm"], marker=myplt.m_dict["cesm"])
ax[4].scatter(delta_P_lm, term2_lm, color=myplt.c_dict["cesm"], s=myplt.s_dict["cesm"], marker=myplt.m_dict["cesm"])
ax[5].scatter(delta_P_lm, term3_lm, color=myplt.c_dict["cesm"], s=myplt.s_dict["cesm"], marker=myplt.m_dict["cesm"])
ax[6].scatter(delta_P_lm, term4_lm, color=myplt.c_dict["cesm"], s=myplt.s_dict["cesm"], marker=myplt.m_dict["cesm"])
ax[7].scatter(delta_P_lm, term5_lm, color=myplt.c_dict["cesm"], s=myplt.s_dict["cesm"], marker=myplt.m_dict["cesm"])

min_val = min(ax[0].get_xlim()[0], ax[0].get_ylim()[0])
max_val = max(ax[0].get_xlim()[1], ax[0].get_ylim()[1])
for i, name in enumerate(["delta_E", "allterms", "delta_E_allterms", "term1", "term2", "term3", "term4", "term5"]):
    ax[i].plot([min_val, max_val], [min_val, max_val], '--', color="gray", zorder=0)
    ax[i].set_xlim(min_val, max_val)
    ax[i].set_ylim(min_val, max_val)
    ax[i].set_xlabel("actual $\delta P_L$ [W m$^{2}$]", fontsize=10)
    ax[i].text(x=0.8, y=0.8, s="1:1 line", color="gray", rotation=45, transform=ax[i].transAxes, ha="center", va="bottom")

    ax[i].plot(delta_P_lm.sortby(delta_P_lm), delta_P_lm.sortby(delta_P_lm) * one_to_one_delta_P[name]["slope"] + one_to_one_delta_P[name]["intercept"], color=myplt.c_dict["cesm"])

    ax[i].text(x=0.05, y=0.95,
        #    s=f"slope = ${one_to_one_delta_P[name]['slope']:0.2f}$\nintercept = ${one_to_one_delta_P[name]['intercept']:0.4f}$ W m$^{{2}}$\nr = ${one_to_one_delta_P[name]['rvalue']:0.2f}$\nr$^2$ = ${one_to_one_delta_P[name]['rvalue']**2:0.2f}$\nRMSE = ${one_to_one_delta_P[name]['rmse']:0.2f}$ W m$^{{2}}$\np = ${one_to_one_delta_P[name]['pvalue']:0.3f}$",
           s=f"slope = ${one_to_one_delta_P[name]['slope']:0.2f}$\nr = ${one_to_one_delta_P[name]['rvalue']:0.2f}$\nr$^2$ = ${one_to_one_delta_P[name]['rvalue']**2:0.2f}$\nRMSE = ${one_to_one_delta_P[name]['rmse']:0.2f}$ W m$^{{2}}$\np = ${one_to_one_delta_P[name]['pvalue']:0.3f}$",
           color=myplt.c_dict["cesm"],
           transform=ax[i].transAxes, ha="left", va="top")

ax[0].set_ylabel("predicted $\delta P_L$ [W m$^{2}$]", fontsize=10)
ax[4].set_ylabel("predicted $\delta P_L$ [W m$^{2}$]", fontsize=10)

ax[0].set_title("$\\delta E_L$", fontsize=10, loc="left")
ax[1].set_title("$\\delta (P-E)_L$", fontsize=10, loc="left")
ax[2].set_title("$\\delta E_L + \\delta (P-E)_L$", fontsize=10, loc="left")

ax[3].set_title("$\\alpha_S \\delta T_S (P-E)$", fontsize=10, loc="left")
ax[4].set_title("$(\\delta \\mathcal{H}_S / \\mathcal{H}_S) (P-E)$", fontsize=10, loc="left")
ax[5].set_title("$-\\mathbf{F}\\cdot \\nabla (\\alpha_S \\delta T_S)$", fontsize=10, loc="left")
ax[6].set_title("$-\\mathbf{F}\\cdot \\nabla (\\mathcal{H}_S / \\mathcal{H}_S)$", fontsize=10, loc="left")
ax[7].set_title("$- \\frac{1}{g} \\nabla \\cdot \\int_0^{p_s} q \, \\delta \\overline{\\mathbf{u}} \, dp$", fontsize=10, loc="left")

# ## Save figure
# plt.show()
# fig.savefig(f'delta_pme_scaling_scatter.svg', bbox_inches='tight')
# fig.savefig(f'delta_pme_scaling_scatter.pdf', bbox_inches='tight')

## S12: Precipitation change from the compositing scheme plotted as the slope of actual and predicted \delta P over land at different latitude bands

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(9, 9), layout="tight", sharey=True, sharex=True)
ax = axes.flatten()

label = ["CESM2 PI", "HadCM3C Hist", "HadCM3C A1B"]

for j, region in enumerate(["global", "trop", "nhmid", "nh"]):

    x = np.array([0, 1.25, 2.25, 3.25])
    yaxis = [
        "Ocean $\\mathcal{H}_{S}$\ncontribution",
        "Land $\\mathcal{H}_{S}$\ncontribution",
        "Approximate\nscaling",
        "Full scaling"
    ][::-1]
    yaxis_math = [
        "$-a_1 \\delta \\mathcal{H}_{S,O} \, P_O$",
        "$a_1 \\delta \\mathcal{H}_{S,L} \, P_O$",
        # "$a_1 \\delta \\mathcal{H}_{S,L} \, P_O - a_1 \\delta \\mathcal{H}_{S,O} \, P_O$",
        "$a_1 (\\delta \\mathcal{H}_{S,L} - \\delta \\mathcal{H}_{S,O}) P_O$",
        "$a_1 \\delta [(\\mathcal{H}_{S,L} - \\mathcal{H}_{S,O}) \, P_O]$",
    ][::-1]

    m = np.zeros((3, 4))
    r2 = np.zeros((3, 4))
    rmse = np.zeros((3, 4))
    for i, (key, item) in enumerate(delta_p_scaling[region].items()):
        m[i] = np.array([
            delta_p_scaling[region][key]["delta_rh_o_term"]["slope"],
            delta_p_scaling[region][key]["delta_rh_l_term"]["slope"],
            delta_p_scaling[region][key]["full_approx"]["slope"],
            delta_p_scaling[region][key]["full"]["slope"],
        ])[::-1]

        r2[i] = np.array([
            delta_p_scaling[region][key]["delta_rh_o_term"]["r_value"]**2,
            delta_p_scaling[region][key]["delta_rh_l_term"]["r_value"]**2,
            delta_p_scaling[region][key]["full_approx"]["r_value"]**2,
            delta_p_scaling[region][key]["full"]["r_value"]**2,
        ])[::-1]

        rmse[i] = np.array([
            delta_p_scaling[region][key]["delta_rh_o_term"]["rmse"],
            delta_p_scaling[region][key]["delta_rh_l_term"]["rmse"],
            delta_p_scaling[region][key]["full_approx"]["rmse"],
            delta_p_scaling[region][key]["full"]["rmse"],
        ])[::-1]

        ax[j].barh(
                x+i/4,
                m[i],
                height=0.25,
                color=myplt.c_dict[key],
                label=label[i],
                alpha=0.75
            )

    # for i in range(4):
    #     ax[j].text(-0.7, x[i]+0.25, yaxis_math[i], ha="center", va="center")

    ax[j].set_yticks(x+0.25)
    if j == 0 or j == 2:
        ax[j].set_yticklabels(yaxis, ha="right", fontsize=10, rotation=20)

    ax[j].legend(frameon=True)

    ax[j].set_xticks([-1.0, 0.0, 1.0, 2.0])
    ax[j].set_xticklabels(["$-$1", "0", "1", "2"], fontsize=10)
    ax[j].set_xlim(-1.4, 2.3)
    # ax[j].axvline(2, xmin, xmax, color="k", ls=":", lw=plt.rcParams["axes.linewidth"])
    ax[j].axvline(1, xmin, xmax, color="k", ls=(0, (5, 5)), lw=plt.rcParams["axes.linewidth"])
    ax[j].axvline(0, xmin, xmax, color="k", ls="-", lw=plt.rcParams["axes.linewidth"])
    # ax[j].axvline(-1, xmin, xmax, color="k", ls=":", lw=plt.rcParams["axes.linewidth"])

    ax[j].set_title(f"[{lat_bounds[region][0]}N, {lat_bounds[region][1]}N]", fontsize=12, fontweight="bold", loc="center")
    ax[j].text(x=0.09, y=0.96, s=f"({myplt.figenum[j]})", fontsize=14, transform=ax[j].transAxes, ha="right", va="top")

for j in range(2,4):
    trans = ax[j].get_xaxis_transform()
    ax[j].annotate("no prediction\nskill", xy=(0,-0.08), xytext=(0,-0.2), ha="center", va="top", xycoords=trans, arrowprops=dict(arrowstyle='-', color='black'))
    ax[j].annotate("perfect\nprediction", xy=(1,-0.08), xytext=(1,-0.2), ha="center", va="top", xycoords=trans, arrowprops=dict(arrowstyle='-', color='black'))
    ax[j].annotate("", xy=(1,-0.12), xytext=(0,-0.12), xycoords=trans, arrowprops=dict(arrowstyle='<->', color='black'))
    ax[j].annotate("", xy=(0,-0.12), xytext=(-1.4,-0.12), xycoords=trans, arrowprops=dict(arrowstyle='<-', color='black'))
    ax[j].annotate("", xy=(1,-0.12), xytext=(2.3,-0.12), xycoords=trans, arrowprops=dict(arrowstyle='<-', color='black'))
    ax[j].text(x=0.5, y=-0.15, s="underpredicts", ha="center", va="center", fontsize=8, transform=trans)
    ax[j].text(x=-0.7, y=-0.15, s="predicts wrong sign", ha="center", va="center", fontsize=8, transform=trans)
    ax[j].text(x=1.65, y=-0.15, s="overpredicts", ha="center", va="center", fontsize=8, transform=trans)

    # ax[j].text(x=0.5, y=-0.3, s="$\\mathbf{slope(}$ predicted$_{\\delta P}$  actual$_{\\delta P}$ $\\mathbf{)}$", ha="center", va="top", fontsize=12, transform=a.transAxes)
    ax[j].text(x=0.5, y=-0.3, s="$\\mathbf{slope(}$ $\\delta P_{\\text{predicted}}$ ; $\\delta P_{\\text{actual}}$ $\\mathbf{)}$", ha="center", va="top", fontsize=12, transform=ax[j].transAxes)

fig.savefig("comp_scheme_delta_p_scaling_slope.pdf")

## S13: One-to-one scatter plots of actual and predicted \delta P over tropical land from compositing scheme for the CESM2 PI PPE

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(8.5, 10), sharey=True, sharex=True, layout="constrained", gridspec_kw={"hspace":0.04, "wspace":0.04})
ax = axs.flatten()

region = "trop"

i = 0
ax[i].scatter(delta_areamean_lat[region]["cesm"]["P_L"], delta_p_scaling[region]["cesm"]["delta_rh_l_term"]["data"], color=myplt.c_dict["cesm"], s=myplt.s_dict["cesm"], marker=myplt.m_dict["cesm"])
ax[i+1].scatter(delta_areamean_lat[region]["hadhst"]["P_L"], delta_p_scaling[region]["hadhst"]["delta_rh_l_term"]["data"], color=myplt.c_dict["hadhst"], s=myplt.s_dict["hadhst"], marker=myplt.m_dict["hadhst"])
ax[i+2].scatter(delta_areamean_lat[region]["hada1b"]["P_L"], delta_p_scaling[region]["hada1b"]["delta_rh_l_term"]["data"], color=myplt.c_dict["hada1b"], s=myplt.s_dict["hada1b"], marker=myplt.m_dict["hada1b"])

ax[i].plot(delta_areamean_lat[region]["cesm"]["P_L"].sortby(delta_areamean_lat[region]["cesm"]["P_L"]), delta_areamean_lat[region]["cesm"]["P_L"].sortby(delta_areamean_lat[region]["cesm"]["P_L"]) * delta_p_scaling[region]["cesm"]["delta_rh_l_term"]['slope'] + delta_p_scaling[region]["cesm"]["delta_rh_l_term"]['intercept'], color=myplt.c_dict["cesm"])
ax[i+1].plot(delta_areamean_lat[region]["hadhst"]["P_L"].sortby(delta_areamean_lat[region]["hadhst"]["P_L"]), delta_areamean_lat[region]["hadhst"]["P_L"].sortby(delta_areamean_lat[region]["hadhst"]["P_L"]) * delta_p_scaling[region]["hadhst"]["delta_rh_l_term"]['slope'] + delta_p_scaling[region]["hadhst"]["delta_rh_l_term"]['intercept'], color=myplt.c_dict["hadhst"])
ax[i+2].plot(delta_areamean_lat[region]["hada1b"]["P_L"].sortby(delta_areamean_lat[region]["hada1b"]["P_L"]), delta_areamean_lat[region]["hada1b"]["P_L"].sortby(delta_areamean_lat[region]["hada1b"]["P_L"]) * delta_p_scaling[region]["hada1b"]["delta_rh_l_term"]['slope'] + delta_p_scaling[region]["hada1b"]["delta_rh_l_term"]['intercept'], color=myplt.c_dict["hada1b"])

i = 3
ax[i].scatter(delta_areamean_lat[region]["cesm"]["P_L"], delta_p_scaling[region]["cesm"]["delta_rh_o_term"]["data"], color=myplt.c_dict["cesm"], s=myplt.s_dict["cesm"], marker=myplt.m_dict["cesm"])
ax[i+1].scatter(delta_areamean_lat[region]["hadhst"]["P_L"], delta_p_scaling[region]["hadhst"]["delta_rh_o_term"]["data"], color=myplt.c_dict["hadhst"], s=myplt.s_dict["hadhst"], marker=myplt.m_dict["hadhst"])
ax[i+2].scatter(delta_areamean_lat[region]["hada1b"]["P_L"], delta_p_scaling[region]["hada1b"]["delta_rh_o_term"]["data"], color=myplt.c_dict["hada1b"], s=myplt.s_dict["hada1b"], marker=myplt.m_dict["hada1b"])

ax[i].plot(delta_areamean_lat[region]["cesm"]["P_L"].sortby(delta_areamean_lat[region]["cesm"]["P_L"]), delta_areamean_lat[region]["cesm"]["P_L"].sortby(delta_areamean_lat[region]["cesm"]["P_L"]) * delta_p_scaling[region]["cesm"]["delta_rh_o_term"]['slope'] + delta_p_scaling[region]["cesm"]["delta_rh_o_term"]['intercept'], color=myplt.c_dict["cesm"])
ax[i+1].plot(delta_areamean_lat[region]["hadhst"]["P_L"].sortby(delta_areamean_lat[region]["hadhst"]["P_L"]), delta_areamean_lat[region]["hadhst"]["P_L"].sortby(delta_areamean_lat[region]["hadhst"]["P_L"]) * delta_p_scaling[region]["hadhst"]["delta_rh_o_term"]['slope'] + delta_p_scaling[region]["hadhst"]["delta_rh_o_term"]['intercept'], color=myplt.c_dict["hadhst"])
ax[i+2].plot(delta_areamean_lat[region]["hada1b"]["P_L"].sortby(delta_areamean_lat[region]["hada1b"]["P_L"]), delta_areamean_lat[region]["hada1b"]["P_L"].sortby(delta_areamean_lat[region]["hada1b"]["P_L"]) * delta_p_scaling[region]["hada1b"]["delta_rh_o_term"]['slope'] + delta_p_scaling[region]["hada1b"]["delta_rh_o_term"]['intercept'], color=myplt.c_dict["hada1b"])

i = 6
ax[i].scatter(delta_areamean_lat[region]["cesm"]["P_L"], delta_p_scaling[region]["cesm"]["full_approx"]["data"], color=myplt.c_dict["cesm"], s=myplt.s_dict["cesm"], marker=myplt.m_dict["cesm"])
ax[i+1].scatter(delta_areamean_lat[region]["hadhst"]["P_L"], delta_p_scaling[region]["hadhst"]["full_approx"]["data"], color=myplt.c_dict["hadhst"], s=myplt.s_dict["hadhst"], marker=myplt.m_dict["hadhst"])
ax[i+2].scatter(delta_areamean_lat[region]["hada1b"]["P_L"], delta_p_scaling[region]["hada1b"]["full_approx"]["data"], color=myplt.c_dict["hada1b"], s=myplt.s_dict["hada1b"], marker=myplt.m_dict["hada1b"])

ax[i].plot(delta_areamean_lat[region]["cesm"]["P_L"].sortby(delta_areamean_lat[region]["cesm"]["P_L"]), delta_areamean_lat[region]["cesm"]["P_L"].sortby(delta_areamean_lat[region]["cesm"]["P_L"]) * delta_p_scaling[region]["cesm"]["full_approx"]['slope'] + delta_p_scaling[region]["cesm"]["full_approx"]['intercept'], color=myplt.c_dict["cesm"])
ax[i+1].plot(delta_areamean_lat[region]["hadhst"]["P_L"].sortby(delta_areamean_lat[region]["hadhst"]["P_L"]), delta_areamean_lat[region]["hadhst"]["P_L"].sortby(delta_areamean_lat[region]["hadhst"]["P_L"]) * delta_p_scaling[region]["hadhst"]["full_approx"]['slope'] + delta_p_scaling[region]["hadhst"]["full_approx"]['intercept'], color=myplt.c_dict["hadhst"])
ax[i+2].plot(delta_areamean_lat[region]["hada1b"]["P_L"].sortby(delta_areamean_lat[region]["hada1b"]["P_L"]), delta_areamean_lat[region]["hada1b"]["P_L"].sortby(delta_areamean_lat[region]["hada1b"]["P_L"]) * delta_p_scaling[region]["hada1b"]["full_approx"]['slope'] + delta_p_scaling[region]["hada1b"]["full_approx"]['intercept'], color=myplt.c_dict["hada1b"])

i = 9
ax[i].scatter(delta_areamean_lat[region]["cesm"]["P_L"], delta_p_scaling[region]["cesm"]["full"]["data"], color=myplt.c_dict["cesm"], s=myplt.s_dict["cesm"], marker=myplt.m_dict["cesm"])
ax[i+1].scatter(delta_areamean_lat[region]["hadhst"]["P_L"], delta_p_scaling[region]["hadhst"]["full"]["data"], color=myplt.c_dict["hadhst"], s=myplt.s_dict["hadhst"], marker=myplt.m_dict["hadhst"])
ax[i+2].scatter(delta_areamean_lat[region]["hada1b"]["P_L"], delta_p_scaling[region]["hada1b"]["full"]["data"], color=myplt.c_dict["hada1b"], s=myplt.s_dict["hada1b"], marker=myplt.m_dict["hada1b"])

ax[i].plot(delta_areamean_lat[region]["cesm"]["P_L"].sortby(delta_areamean_lat[region]["cesm"]["P_L"]), delta_areamean_lat[region]["cesm"]["P_L"].sortby(delta_areamean_lat[region]["cesm"]["P_L"]) * delta_p_scaling[region]["cesm"]["full"]['slope'] + delta_p_scaling[region]["cesm"]["full"]['intercept'], color=myplt.c_dict["cesm"])
ax[i+1].plot(delta_areamean_lat[region]["hadhst"]["P_L"].sortby(delta_areamean_lat[region]["hadhst"]["P_L"]), delta_areamean_lat[region]["hadhst"]["P_L"].sortby(delta_areamean_lat[region]["hadhst"]["P_L"]) * delta_p_scaling[region]["hadhst"]["full"]['slope'] + delta_p_scaling[region]["hadhst"]["full"]['intercept'], color=myplt.c_dict["hadhst"])
ax[i+2].plot(delta_areamean_lat[region]["hada1b"]["P_L"].sortby(delta_areamean_lat[region]["hada1b"]["P_L"]), delta_areamean_lat[region]["hada1b"]["P_L"].sortby(delta_areamean_lat[region]["hada1b"]["P_L"]) * delta_p_scaling[region]["hada1b"]["full"]['slope'] + delta_p_scaling[region]["hada1b"]["full"]['intercept'], color=myplt.c_dict["hada1b"])

for i, (a, name, kind) in enumerate(zip(ax, ["cesm", "hadhst", "hada1b"]*4, ["delta_rh_l_term"]*3 + ["delta_rh_o_term"]*3 + ["full_approx"]*3 + ["full"]*3)):
    a.set_xlim(-0.2, 0.5)
    a.set_ylim(-0.2, 0.5)
    a.plot([-0.2, 0.5], [-0.2, 0.5], ls="--", lw=0.8, color="gray", zorder=0)
    a.text(x=0.9, y=0.9, s="1:1 line", color="gray", rotation=45, transform=a.transAxes, ha="center", va="top")
    a.text(x=0.05, y=0.95,
            s=f"slope = ${delta_p_scaling[region][name][kind]['slope']:0.2f}$\nintercept = ${delta_p_scaling[region][name][kind]['intercept']:0.4f}$ mm day$^{{-1}}$\nr = ${delta_p_scaling[region][name][kind]['r_value']:0.2f}$\nr$^2$ = ${delta_p_scaling[region][name][kind]['r_value']**2:0.2f}$\n$p$ = {delta_p_scaling[region][name][kind]['p_value']:0.3e}\nRMSE = ${delta_p_scaling[region][name][kind]['rmse']:0.3f}$ mm day$^{{-1}}$",
            color=myplt.c_dict[name], fontsize=6,
            transform=a.transAxes, ha="left", va="top")

for a in ax[-3:]:
    a.set_xlabel("Actual $\delta P_L$ [mm day$^{-1}$]", fontsize=10)
    
ax[0].set_title(f"CESM2 PI PPE ($a_1={lr[region]['cesm']['lr'].slope:0.3f}$)", loc="center", fontweight="bold")
ax[1].set_title(f"HadCM3C Hist PPE ($a_1={lr[region]['hadhst']['lr'].slope:0.3f}$)", loc="center", fontweight="bold")
ax[2].set_title(f"HadCM3C A1B PPE ($a_1={lr[region]['hada1b']['lr'].slope:0.3f}$)", loc="center", fontweight="bold")

ax[0].set_ylabel("$\mathbf{Land}$ $\\boldsymbol{\\mathcal{H}}_{S}$ $\mathbf{contribution:}$\n$a_1 \\delta \\mathcal{H}_{S,L} \, P_O$\n[mm day$^{-1}$]", fontsize=10)
ax[3].set_ylabel("$\mathbf{Ocean}$ $\\boldsymbol{\\mathcal{H}}_{S}$ $\mathbf{contribution:}$\n$- a_1 \\delta \\mathcal{H}_{S,O} \, P_O$\n[mm day$^{-1}$]", fontsize=10)
ax[6].set_ylabel("$\mathbf{Approx.}$ $\mathbf{scaling:}$\n$a_1 \\delta \\mathcal{H}_{S,L} \, P_O - a_1 \\delta \\mathcal{H}_{S,O} \, P_O$\n[mm day$^{-1}$]", fontsize=10)
ax[9].set_ylabel("$\mathbf{Full}$ $\mathbf{scaling:}$\n$a_1 \\delta (\\mathcal{H}_{S,L} \, P_O - \\mathcal{H}_{S,O} \, P_O)$\n[mm day$^{-1}$]", fontsize=10)

fig.suptitle("TROPICAL [30S, 30N] with $a_1$ from regression across PPE", fontsize=12, x=0.525, ha="center")

plt.savefig(f"comp_scheme_delta_p_scaling_scatter_{region}.pdf", dpi=300, bbox_inches="tight")

## S14: One-to-one scatter plots of actual and predicted \delta P over global land from compositing scheme for the CESM2 PI PPE

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(8.5, 10), sharey=True, sharex=True, layout="constrained", gridspec_kw={"hspace":0.04, "wspace":0.04})
ax = axs.flatten()

region = "global"

i = 0
ax[i].scatter(delta_areamean_lat[region]["cesm"]["P_L"], delta_p_scaling[region]["cesm"]["delta_rh_l_term"]["data"], color=myplt.c_dict["cesm"], s=myplt.s_dict["cesm"], marker=myplt.m_dict["cesm"])
ax[i+1].scatter(delta_areamean_lat[region]["hadhst"]["P_L"], delta_p_scaling[region]["hadhst"]["delta_rh_l_term"]["data"], color=myplt.c_dict["hadhst"], s=myplt.s_dict["hadhst"], marker=myplt.m_dict["hadhst"])
ax[i+2].scatter(delta_areamean_lat[region]["hada1b"]["P_L"], delta_p_scaling[region]["hada1b"]["delta_rh_l_term"]["data"], color=myplt.c_dict["hada1b"], s=myplt.s_dict["hada1b"], marker=myplt.m_dict["hada1b"])

ax[i].plot(delta_areamean_lat[region]["cesm"]["P_L"].sortby(delta_areamean_lat[region]["cesm"]["P_L"]), delta_areamean_lat[region]["cesm"]["P_L"].sortby(delta_areamean_lat[region]["cesm"]["P_L"]) * delta_p_scaling[region]["cesm"]["delta_rh_l_term"]['slope'] + delta_p_scaling[region]["cesm"]["delta_rh_l_term"]['intercept'], color=myplt.c_dict["cesm"])
ax[i+1].plot(delta_areamean_lat[region]["hadhst"]["P_L"].sortby(delta_areamean_lat[region]["hadhst"]["P_L"]), delta_areamean_lat[region]["hadhst"]["P_L"].sortby(delta_areamean_lat[region]["hadhst"]["P_L"]) * delta_p_scaling[region]["hadhst"]["delta_rh_l_term"]['slope'] + delta_p_scaling[region]["hadhst"]["delta_rh_l_term"]['intercept'], color=myplt.c_dict["hadhst"])
ax[i+2].plot(delta_areamean_lat[region]["hada1b"]["P_L"].sortby(delta_areamean_lat[region]["hada1b"]["P_L"]), delta_areamean_lat[region]["hada1b"]["P_L"].sortby(delta_areamean_lat[region]["hada1b"]["P_L"]) * delta_p_scaling[region]["hada1b"]["delta_rh_l_term"]['slope'] + delta_p_scaling[region]["hada1b"]["delta_rh_l_term"]['intercept'], color=myplt.c_dict["hada1b"])

i = 3
ax[i].scatter(delta_areamean_lat[region]["cesm"]["P_L"], delta_p_scaling[region]["cesm"]["delta_rh_o_term"]["data"], color=myplt.c_dict["cesm"], s=myplt.s_dict["cesm"], marker=myplt.m_dict["cesm"])
ax[i+1].scatter(delta_areamean_lat[region]["hadhst"]["P_L"], delta_p_scaling[region]["hadhst"]["delta_rh_o_term"]["data"], color=myplt.c_dict["hadhst"], s=myplt.s_dict["hadhst"], marker=myplt.m_dict["hadhst"])
ax[i+2].scatter(delta_areamean_lat[region]["hada1b"]["P_L"], delta_p_scaling[region]["hada1b"]["delta_rh_o_term"]["data"], color=myplt.c_dict["hada1b"], s=myplt.s_dict["hada1b"], marker=myplt.m_dict["hada1b"])

ax[i].plot(delta_areamean_lat[region]["cesm"]["P_L"].sortby(delta_areamean_lat[region]["cesm"]["P_L"]), delta_areamean_lat[region]["cesm"]["P_L"].sortby(delta_areamean_lat[region]["cesm"]["P_L"]) * delta_p_scaling[region]["cesm"]["delta_rh_o_term"]['slope'] + delta_p_scaling[region]["cesm"]["delta_rh_o_term"]['intercept'], color=myplt.c_dict["cesm"])
ax[i+1].plot(delta_areamean_lat[region]["hadhst"]["P_L"].sortby(delta_areamean_lat[region]["hadhst"]["P_L"]), delta_areamean_lat[region]["hadhst"]["P_L"].sortby(delta_areamean_lat[region]["hadhst"]["P_L"]) * delta_p_scaling[region]["hadhst"]["delta_rh_o_term"]['slope'] + delta_p_scaling[region]["hadhst"]["delta_rh_o_term"]['intercept'], color=myplt.c_dict["hadhst"])
ax[i+2].plot(delta_areamean_lat[region]["hada1b"]["P_L"].sortby(delta_areamean_lat[region]["hada1b"]["P_L"]), delta_areamean_lat[region]["hada1b"]["P_L"].sortby(delta_areamean_lat[region]["hada1b"]["P_L"]) * delta_p_scaling[region]["hada1b"]["delta_rh_o_term"]['slope'] + delta_p_scaling[region]["hada1b"]["delta_rh_o_term"]['intercept'], color=myplt.c_dict["hada1b"])

i = 6
ax[i].scatter(delta_areamean_lat[region]["cesm"]["P_L"], delta_p_scaling[region]["cesm"]["full_approx"]["data"], color=myplt.c_dict["cesm"], s=myplt.s_dict["cesm"], marker=myplt.m_dict["cesm"])
ax[i+1].scatter(delta_areamean_lat[region]["hadhst"]["P_L"], delta_p_scaling[region]["hadhst"]["full_approx"]["data"], color=myplt.c_dict["hadhst"], s=myplt.s_dict["hadhst"], marker=myplt.m_dict["hadhst"])
ax[i+2].scatter(delta_areamean_lat[region]["hada1b"]["P_L"], delta_p_scaling[region]["hada1b"]["full_approx"]["data"], color=myplt.c_dict["hada1b"], s=myplt.s_dict["hada1b"], marker=myplt.m_dict["hada1b"])

ax[i].plot(delta_areamean_lat[region]["cesm"]["P_L"].sortby(delta_areamean_lat[region]["cesm"]["P_L"]), delta_areamean_lat[region]["cesm"]["P_L"].sortby(delta_areamean_lat[region]["cesm"]["P_L"]) * delta_p_scaling[region]["cesm"]["full_approx"]['slope'] + delta_p_scaling[region]["cesm"]["full_approx"]['intercept'], color=myplt.c_dict["cesm"])
ax[i+1].plot(delta_areamean_lat[region]["hadhst"]["P_L"].sortby(delta_areamean_lat[region]["hadhst"]["P_L"]), delta_areamean_lat[region]["hadhst"]["P_L"].sortby(delta_areamean_lat[region]["hadhst"]["P_L"]) * delta_p_scaling[region]["hadhst"]["full_approx"]['slope'] + delta_p_scaling[region]["hadhst"]["full_approx"]['intercept'], color=myplt.c_dict["hadhst"])
ax[i+2].plot(delta_areamean_lat[region]["hada1b"]["P_L"].sortby(delta_areamean_lat[region]["hada1b"]["P_L"]), delta_areamean_lat[region]["hada1b"]["P_L"].sortby(delta_areamean_lat[region]["hada1b"]["P_L"]) * delta_p_scaling[region]["hada1b"]["full_approx"]['slope'] + delta_p_scaling[region]["hada1b"]["full_approx"]['intercept'], color=myplt.c_dict["hada1b"])

i = 9
ax[i].scatter(delta_areamean_lat[region]["cesm"]["P_L"], delta_p_scaling[region]["cesm"]["full"]["data"], color=myplt.c_dict["cesm"], s=myplt.s_dict["cesm"], marker=myplt.m_dict["cesm"])
ax[i+1].scatter(delta_areamean_lat[region]["hadhst"]["P_L"], delta_p_scaling[region]["hadhst"]["full"]["data"], color=myplt.c_dict["hadhst"], s=myplt.s_dict["hadhst"], marker=myplt.m_dict["hadhst"])
ax[i+2].scatter(delta_areamean_lat[region]["hada1b"]["P_L"], delta_p_scaling[region]["hada1b"]["full"]["data"], color=myplt.c_dict["hada1b"], s=myplt.s_dict["hada1b"], marker=myplt.m_dict["hada1b"])

ax[i].plot(delta_areamean_lat[region]["cesm"]["P_L"].sortby(delta_areamean_lat[region]["cesm"]["P_L"]), delta_areamean_lat[region]["cesm"]["P_L"].sortby(delta_areamean_lat[region]["cesm"]["P_L"]) * delta_p_scaling[region]["cesm"]["full"]['slope'] + delta_p_scaling[region]["cesm"]["full"]['intercept'], color=myplt.c_dict["cesm"])
ax[i+1].plot(delta_areamean_lat[region]["hadhst"]["P_L"].sortby(delta_areamean_lat[region]["hadhst"]["P_L"]), delta_areamean_lat[region]["hadhst"]["P_L"].sortby(delta_areamean_lat[region]["hadhst"]["P_L"]) * delta_p_scaling[region]["hadhst"]["full"]['slope'] + delta_p_scaling[region]["hadhst"]["full"]['intercept'], color=myplt.c_dict["hadhst"])
ax[i+2].plot(delta_areamean_lat[region]["hada1b"]["P_L"].sortby(delta_areamean_lat[region]["hada1b"]["P_L"]), delta_areamean_lat[region]["hada1b"]["P_L"].sortby(delta_areamean_lat[region]["hada1b"]["P_L"]) * delta_p_scaling[region]["hada1b"]["full"]['slope'] + delta_p_scaling[region]["hada1b"]["full"]['intercept'], color=myplt.c_dict["hada1b"])

for i, (a, name, kind) in enumerate(zip(ax, ["cesm", "hadhst", "hada1b"]*4, ["delta_rh_l_term"]*3 + ["delta_rh_o_term"]*3 + ["full_approx"]*3 + ["full"]*3)):
    a.set_xlim(-0.175, 0.4)
    a.set_ylim(-0.175, 0.4)
    a.plot([-0.25, 0.4], [-0.25, 0.4], ls="--", lw=0.8, color="gray", zorder=0)
    a.text(x=0.9, y=0.9, s="1:1 line", color="gray", rotation=45, transform=a.transAxes, ha="center", va="top")
    a.text(x=0.05, y=0.95,
            s=f"slope = ${delta_p_scaling[region][name][kind]['slope']:0.2f}$\nintercept = ${delta_p_scaling[region][name][kind]['intercept']:0.4f}$ mm day$^{{-1}}$\nr = ${delta_p_scaling[region][name][kind]['r_value']:0.2f}$\nr$^2$ = ${delta_p_scaling[region][name][kind]['r_value']**2:0.2f}$\n$p$ = {delta_p_scaling[region][name][kind]['p_value']:0.3e}\nRMSE = ${delta_p_scaling[region][name][kind]['rmse']:0.3f}$ mm day$^{{-1}}$",
            color=myplt.c_dict[name], fontsize=6,
            transform=a.transAxes, ha="left", va="top")

for a in ax[-3:]:
    a.set_xlabel("Actual $\delta P_L$ [mm day$^{-1}$]", fontsize=10)
    
ax[0].set_title(f"CESM2 PI PPE ($a_1={lr[region]['cesm']['lr'].slope:0.3f}$)", loc="center", fontweight="bold")
ax[1].set_title(f"HadCM3C Hist PPE ($a_1={lr[region]['hadhst']['lr'].slope:0.3f}$)", loc="center", fontweight="bold")
ax[2].set_title(f"HadCM3C A1B PPE ($a_1={lr[region]['hada1b']['lr'].slope:0.3f}$)", loc="center", fontweight="bold")

ax[0].set_ylabel("$\mathbf{Land}$ $\\boldsymbol{\\mathcal{H}}_{S}$ $\mathbf{contribution:}$\n$a_1 \\delta \\mathcal{H}_{S,L} \, P_O$\n[mm day$^{-1}$]", fontsize=10)
ax[3].set_ylabel("$\mathbf{Ocean}$ $\\boldsymbol{\\mathcal{H}}_{S}$ $\mathbf{contribution:}$\n$- a_1 \\delta \\mathcal{H}_{S,O} \, P_O$\n[mm day$^{-1}$]", fontsize=10)
ax[6].set_ylabel("$\mathbf{Approx.}$ $\mathbf{scaling:}$\n$a_1 \\delta \\mathcal{H}_{S,L} \, P_O - a_1 \\delta \\mathcal{H}_{S,O} \, P_O$\n[mm day$^{-1}$]", fontsize=10)
ax[9].set_ylabel("$\mathbf{Full}$ $\mathbf{scaling:}$\n$a_1 \\delta (\\mathcal{H}_{S,L} \, P_O - \\mathcal{H}_{S,O} \, P_O)$\n[mm day$^{-1}$]", fontsize=10)

fig.suptitle("GLOBAL [90S, 90N] with $a_1$ from regression across PPE", fontsize=12, x=0.525, ha="center")

plt.savefig(f"comp_scheme_delta_p_scaling_scatter_{region}.pdf", dpi=300, bbox_inches="tight")